In [61]:
import tensorflow as tf
import tensorflow_hub as hub
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [62]:
## Check for CUDA GPU (skip if no CUDA GPU) ##

print(tf.test.is_built_with_cuda()) # Should be True
print(tf.config.list_physical_devices('GPU')) # Should not be empty

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [63]:
### DEEP LEARNING ###
from helper import *

## EXTRACTING X AND Y MATRICES. READ HELPER FOR FUNCTION INFO ##
dh = data_handle()

classes = (2,5,11)
img_size = 128

X, y = dh.featurize(resize=img_size, n=750, include=classes)
num_classes = len(classes)

In [64]:
## Preprocess images ##

lb = preprocessing.LabelBinarizer()

range_ = np.array(list(set(y)))
out = lb.fit(range_)
y_binary = out.transform(y.astype(int))

scalify = preprocessing.StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train = scalify.fit_transform(X_train)
X_test = scalify.fit_transform(X_test)

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X, y_binary, test_size=0.20, random_state=42)
X_train_b = scalify.fit_transform(X_train_b)
X_test_b = scalify.fit_transform(X_test_b)

print(X_train_b.shape, X_test_b.shape)

(1792, 16384) (449, 16384)


In [65]:
## AlexNet variant with adjusted image size ##

alexnet = Sequential([
    Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(img_size,img_size,1)),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [66]:
## Compile and display full network ##

alexnet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
alexnet.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 30, 30, 96)        11712     
_________________________________________________________________
batch_normalization_33 (Batc (None, 30, 30, 96)        384       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 14, 14, 256)       614656    
_________________________________________________________________
batch_normalization_34 (Batc (None, 14, 14, 256)       1024      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 6, 6, 384)        

In [67]:
## Makes a bigass file for saving weights, be aware ##

checkpoint = ModelCheckpoint(filepath='best_weights_alexnet.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [68]:
alexnet.fit(X_train_b.reshape(-1,img_size,img_size,1), y_train_b, steps_per_epoch=50,
           epochs=20, batch_size=64, validation_data=(X_test_b.reshape(-1,img_size,img_size,1), y_test_b), 
           callbacks=[checkpoint, lr_reduce, early_stop])

Epoch 1/20
50/50 [==============================] - 4s 74ms/step - loss: 2.6496 - accuracy: 0.3341 - val_loss: 1.0999 - val_accuracy: 0.3519
Epoch 2/20
50/50 [==============================] - 3s 68ms/step - loss: 1.1080 - accuracy: 0.3663 - val_loss: 1.0839 - val_accuracy: 0.3831
Epoch 3/20
50/50 [==============================] - 3s 56ms/step - loss: 1.0948 - accuracy: 0.4016 - val_loss: 1.1503 - val_accuracy: 0.3541
Epoch 4/20
50/50 [==============================] - 3s 56ms/step - loss: 1.0810 - accuracy: 0.4166 - val_loss: 1.0997 - val_accuracy: 0.3408
Epoch 5/20
50/50 [==============================] - 3s 56ms/step - loss: 1.0156 - accuracy: 0.4762 - val_loss: 1.0876 - val_accuracy: 0.3697
Epoch 6/20
50/50 [==============================] - 3s 56ms/step - loss: 0.9825 - accuracy: 0.5094 - val_loss: 1.0987 - val_accuracy: 0.3608
Epoch 7/20
50/50 [==============================] - 3s 57ms/step - loss: 0.9593 - accuracy: 0.5272 - val_loss: 1.0940 - val_accuracy: 0.3764


In [69]:
## Check how confident prediction on random image is ##

predictions = alexnet.predict(X_test.reshape(-1,img_size,img_size,1))
score = tf.nn.softmax(predictions[0])
print(score)

tf.Tensor([0.30888242 0.34784117 0.3432764 ], shape=(3,), dtype=float32)


In [70]:
alexnet.evaluate(X_test.reshape(-1,img_size,img_size,1), y_test_b)

15/15 [==============================] - 0s 11ms/step - loss: 1.0839 - accuracy: 0.3831


[1.0839285850524902, 0.38307350873947144]

In [71]:
## VGG variant with adjusted image size ##

vgg = Sequential([
    Conv2D(16, 3, activation='relu', padding='same', input_shape=(img_size,img_size,1)),
    MaxPool2D(pool_size=(2,2)),
    SeparableConv2D(32, 3, activation='relu', padding='same'),
    SeparableConv2D(32, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPool2D(),
    SeparableConv2D(64, 3, activation='relu', padding='same'),
    SeparableConv2D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPool2D(),
    SeparableConv2D(128, 3, activation='relu', padding='same'),
    SeparableConv2D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPool2D(),
    SeparableConv2D(256, 3, activation='relu', padding='same'),
    SeparableConv2D(256, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPool2D(),
    SeparableConv2D(512, 3, activation='relu', padding='same'),
    SeparableConv2D(512, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPool2D(),
    Flatten(),
    Dense(1028, activation='relu'),
    BatchNormalization(),
    Dropout(0.9),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.7),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

In [72]:
vgg.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
vgg.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 128, 128, 16)      160       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
separable_conv2d_20 (Separab (None, 64, 64, 32)        688       
_________________________________________________________________
separable_conv2d_21 (Separab (None, 64, 64, 32)        1344      
_________________________________________________________________
batch_normalization_38 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
separable_conv2d_22 (Separab (None, 32, 32, 64)       

In [73]:
## Makes a bigass file for saving weights, be aware ##

checkpoint = ModelCheckpoint(filepath='best_weights_vgg_like.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [74]:
vgg.fit(X_train_b.reshape(-1,img_size,img_size,1), y_train_b, steps_per_epoch=50,
           epochs=20, batch_size=64, validation_data=(X_test_b.reshape(-1,img_size,img_size,1), y_test_b), 
           callbacks=[checkpoint, lr_reduce, early_stop])

Epoch 1/20
50/50 [==============================] - 7s 149ms/step - loss: 1.6195 - accuracy: 0.3487 - val_loss: 1.1046 - val_accuracy: 0.3163
Epoch 2/20
50/50 [==============================] - 7s 136ms/step - loss: 1.4503 - accuracy: 0.3325 - val_loss: 1.1126 - val_accuracy: 0.3163
Epoch 3/20
50/50 [==============================] - 7s 137ms/step - loss: 1.3304 - accuracy: 0.3378 - val_loss: 1.1166 - val_accuracy: 0.3163
Epoch 4/20
50/50 [==============================] - 7s 137ms/step - loss: 1.3009 - accuracy: 0.3269 - val_loss: 1.1166 - val_accuracy: 0.3163
Epoch 5/20
50/50 [==============================] - 7s 136ms/step - loss: 1.2924 - accuracy: 0.3250 - val_loss: 1.1168 - val_accuracy: 0.3163
Epoch 6/20
50/50 [==============================] - 7s 137ms/step - loss: 1.2900 - accuracy: 0.3391 - val_loss: 1.1181 - val_accuracy: 0.3163


In [75]:
vgg.evaluate(X_test_b.reshape(-1,img_size,img_size,1), y_test_b)

15/15 [==============================] - 0s 17ms/step - loss: 1.1046 - accuracy: 0.3163


[1.1045939922332764, 0.31625834107398987]

In [81]:
X_train_b_rgb = np.zeros((len(X_train_b), img_size, img_size, 3))
X_test_b_rgb = np.zeros((len(X_test_b), img_size, img_size, 3))

for i in range(len(X_train_b)):
    X_train_b_rgb[i] = cv2.cvtColor(X_train_b[i],cv2.COLOR_GRAY2RGB).reshape(-1, img_size, img_size, 3)
for i in range(len(X_test_b)):
    X_test_b_rgb[i] = cv2.cvtColor(X_test_b[i],cv2.COLOR_GRAY2RGB).reshape(-1, img_size, img_size, 3)

In [82]:
## Transfer learning ##

classifier_model ="https://tfhub.dev/google/imagenet/resnet_v2_101/classification/4"
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

# classifier = Sequential([
#     hub.KerasLayer(classifier_model, input_shape=(img_size, img_size, 1))
# ])

# feature_extractor_layer = hub.KerasLayer(
#     feature_extractor_model, input_shape=(img_size, img_size, 1), trainable=False)

# classifier.summary()

from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input
from keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(128, 128, 3)))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
              
preprocess_input(X_train_b_rgb)
preprocess_input(X_test_b_rgb)

array([[[[-103.25762265, -116.09762265, -122.99862265],
         [-103.35187851, -116.19187851, -123.09287851],
         [-103.50867387, -116.34867387, -123.24967387],
         ...,
         [-103.73514629, -116.57514629, -123.47614629],
         [-103.8273211 , -116.6673211 , -123.5683211 ],
         [-103.85495468, -116.69495468, -123.59595468]],

        [[-103.05520605, -115.89520605, -122.79620605],
         [-103.23162853, -116.07162853, -122.97262853],
         [-103.34960591, -116.18960591, -123.09060591],
         ...,
         [-103.54413088, -116.38413088, -123.28513088],
         [-103.54238575, -116.38238575, -123.28338575],
         [-103.65389891, -116.49389891, -123.39489891]],

        [[-103.03854494, -115.87854494, -122.77954494],
         [-103.07373284, -115.91373284, -122.81473284],
         [-103.25773316, -116.09773316, -122.99873316],
         ...,
         [-103.79431469, -116.63431469, -123.53531469],
         [-103.94573345, -116.78573345, -123.68673345],
  

In [83]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [84]:
## Makes a bigass file for saving weights, be aware ##

checkpoint = ModelCheckpoint(filepath='best_weights_resnet.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [88]:
model.fit(X_train_b_rgb, y_train_b, steps_per_epoch=50,
         epochs=20, batch_size=64, validation_data=(X_test_b_rgb, y_test_b),
         callbacks=[checkpoint, lr_reduce, early_stop])

Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[64,64,32,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node functional_9/conv2_block1_1_bn/FusedBatchNormV3 (defined at <ipython-input-85-305c96e2f137>:3) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_127824]

Function call stack:
train_function


In [16]:
model.evaluate(X_test_b_rgb, y_test_b)

19/19 [==============================] - 2s 113ms/step - loss: 1.8295 - accuracy: 0.2003


[1.829487919807434, 0.20033389329910278]